# Music Recommendation Engine

### Content-based-recommendation-system

In [ ]:
#Importing spotipy, a lightweight Python library used to access the Spotify Web API.
!pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util

In [ ]:
client_id = #insert-your-id
client_secret= #insert-your-secret-code

In [ ]:
scope = 'user-library-read'

In [ ]:
token = util.prompt_for_user_token(scope, client_id= client_id, client_secret=client_secret, redirect_uri='http://localhost:3000/callback')

In [ ]:
sp = spotipy.Spotify(auth=token)

In [ ]:
playlist_dic = {}
playlist_cover_art = {}
for i in sp.current_user_playlists()['items']:

    playlist_dic[i['name']] = i['uri'].split(':')[2]
    playlist_cover_art[i['uri'].split(':')[2]] = i['images'][0]['url']

In [ ]:
playlist_dic

In [ ]:
def generate_playlist_df(playlist_name, playlist_dic, spotify_data):
    
    playlist = pd.DataFrame()

    for i, j in enumerate(sp.playlist(playlist_dic[playlist_name])['tracks']['items']):
        playlist.loc[i, 'artist'] = j['track']['artists'][0]['name']
        playlist.loc[i, 'track_name'] = j['track']['name']
        playlist.loc[i, 'track_id'] = j['track']['id']
        playlist.loc[i, 'url'] = j['track']['album']['images'][1]['url']
        playlist.loc[i, 'date_added'] = j['added_at']

    playlist['date_added'] = pd.to_datetime(playlist['date_added'])  
    
    playlist = playlist[playlist['track_id'].isin(spotify_data['track_id'].values)].sort_values('date_added',ascending = False)

    return playlist

In [ ]:
# Song's Features Data

In [ ]:
import pandas as pd
spotify_data = pd.read_csv('SpotifyFeatures.csv')

In [ ]:
spotify_data.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [ ]:
name_ply = input("Enter name of playlist : ")

Enter name of playlist : if the whole world was watching i would still dance with you


In [ ]:
#playlist_df = generate_playlist_df('one more dance two more drinks', playlist_dic, spotify_data)
playlist_df = generate_playlist_df(name_ply, playlist_dic, spotify_data)

In [ ]:
playlist_df.head()

,artist,track_name,track_id,url,date_added
12,Conan Gray,Lookalike,4K3gM7kzX4POAfrs71HomN,https://i.scdn.co/image/ab67616d00001e021da538...,2022-06-28 23:45:25+00:00
11,Train,Drops of Jupiter (Tell Me),2hKdd3qO7cWr2Jo0Bcs0MA,https://i.scdn.co/image/ab67616d00001e02a65df7...,2022-05-06 21:12:45+00:00
5,Taylor Swift,Call It What You Want,1GwMQaZz6Au3QLDbjbMdme,https://i.scdn.co/image/ab67616d00001e02da5d5a...,2022-03-26 19:32:53+00:00
1,Taylor Swift,Gorgeous,1ZY1PqizIl78geGM4xWlEA,https://i.scdn.co/image/ab67616d00001e02da5d5a...,2022-03-26 19:10:30+00:00


In [ ]:
#Creating playlist DataFrame

In [ ]:
from skimage import io
import matplotlib.pyplot as plt

def visualize_cover_art(playlist_df):
    temp = playlist_df['url'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))) , facecolor='#8cfc03')
    columns=5
    
    for i, url in enumerate(temp):
        plt.subplot(len(temp) / columns + 1, columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks([])
        plt.yticks([])
        s='' 
        plt.xlabel(s.join(playlist_df['track_name'].values[i].split(' ')[:4]), fontsize = 10, fontweight='bold')
        plt.tight_layout(h_pad=0.8, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()

In [ ]:
#visualize_cover_art(playlist_df)

In [ ]:
playlist_df.to_csv('playlist.csv')

In [ ]:
#Feature Engineering

In [ ]:
spotify_features_df = spotify_data

In [ ]:
genre_OHE = pd.get_dummies(spotify_features_df.genre)
genre_OHE.head()

,A Capella,Alternative,Anime,Blues,Children's Music,Children’s Music,Classical,Comedy,Country,Dance,...,Pop,R&B,Rap,Reggae,Reggaeton,Rock,Ska,Soul,Soundtrack,World
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
key_OHE = pd.get_dummies(spotify_features_df.key)
key_OHE.head()

,A,A#,B,C,C#,D,D#,E,F,F#,G,G#
0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
spotify_features_df = pd.DataFrame(spotify_features_df)

In [ ]:
scaled_features = MinMaxScaler().fit_transform([
    spotify_features_df['acousticness'].values,
    spotify_features_df['danceability'].values,
    spotify_features_df['duration_ms'].values,
    spotify_features_df['energy'].values,
    spotify_features_df['instrumentalness'].values,
    spotify_features_df['liveness'].values,
    spotify_features_df['loudness'].values,
    spotify_features_df['speechiness'].values,
    spotify_features_df['tempo'].values,
    spotify_features_df['valence'].values])

In [ ]:
spotify_features_df[['acousticness','danceability','duration_ms','energy','instrumentalness','liveness','loudness','speechiness','tempo','valence']] = scaled_features.T

In [ ]:
spotify_features_df = spotify_features_df.drop('genre',axis = 1)
spotify_features_df = spotify_features_df.drop('artist_name', axis = 1)
spotify_features_df = spotify_features_df.drop('track_name', axis = 1)
spotify_features_df = spotify_features_df.drop('popularity',axis = 1)
spotify_features_df = spotify_features_df.drop('key', axis = 1)
spotify_features_df = spotify_features_df.drop('mode', axis = 1)
spotify_features_df = spotify_features_df.drop('time_signature', axis = 1)

In [ ]:
spotify_features_df = spotify_features_df.join(genre_OHE)
spotify_features_df = spotify_features_df.join(key_OHE)

In [ ]:
spotify_features_df.shape

(232725, 50)

In [ ]:
spotify_features_df.head()

,track_id,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,...,B,C,C#,D,D#,E,F,F#,G,G#
0,0BRjO6ga9RKCKjfDqeFgWV,0.000025,0.000022,1.0,0.000028,0.000018,0.000022,0.0,0.000019,0.001699,...,0,0,1,0,0,0,0,0,0,0
1,0BjC1NfoEOOusryehmNudP,0.000042,0.000045,1.0,0.000046,0.000040,0.000042,0.0,0.000041,0.001307,...,0,0,0,0,0,0,0,1,0,0
2,0CoSDzoNIKCRs124s9uTVy,0.000087,0.000085,1.0,0.000082,0.000082,0.000082,0.0,0.000082,0.000666,...,0,1,0,0,0,0,0,0,0,0
3,0Gc6TVm52BwZD07Ki6tIvf,0.000084,0.000081,1.0,0.000082,0.000080,0.000081,0.0,0.000080,0.001207,...,0,0,1,0,0,0,0,0,0,0
4,0IuslXpMROHdEPvSl1fTQK,0.000267,0.000260,1.0,0.000259,0.000257,0.000258,0.0,0.000256,0.001957,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
# Generating Playlist Vector

In [ ]:
from datetime import datetime

In [ ]:
def generate_playlist_vector(spotify_features, playlist_df, weight_factor):
    
    spotify_features_playlist = spotify_features[spotify_features['track_id'].isin(playlist_df['track_id'].values)]
    spotify_features_playlist = spotify_features_playlist.merge(playlist_df[['track_id','date_added']], on = 'track_id', how = 'inner')
    
    spotify_features_nonplaylist = spotify_features[~spotify_features['track_id'].isin(playlist_df['track_id'].values)]
    
    playlist_feature_set = spotify_features_playlist.sort_values('date_added',ascending=False)
    
    
    most_recent_date = playlist_feature_set.iloc[0,-1]
    
    for ix, row in playlist_feature_set.iterrows():
        playlist_feature_set.loc[ix,'days_from_recent'] = int((most_recent_date.to_pydatetime() - row.iloc[-1].to_pydatetime()).days)
        
    
    playlist_feature_set['weight'] = playlist_feature_set['days_from_recent'].apply(lambda x: weight_factor ** (-x))
    
    playlist_feature_set_weighted = playlist_feature_set.copy()
    
    playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-3].mul(playlist_feature_set_weighted.weight.astype(int),0))   
    
    playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-3]
    

    
    return playlist_feature_set_weighted_final.sum(axis = 0), spotify_features_nonplaylist

In [ ]:
playlist_vector, nonplaylist_df = generate_playlist_vector(spotify_features_df, playlist_df, 1.2)

In [ ]:
print(playlist_vector.shape)
print(nonplaylist_df.shape)

(50,)
(232717, 50)


In [ ]:
nonplaylist_df.shape

(232717, 50)

In [ ]:
# Generate Recommendations

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def generate_recommendation(spotify_data, playlist_vector, nonplaylist_df):

    non_playlist = spotify_data[spotify_data['track_id'].isin(nonplaylist_df['track_id'].values)]
    non_playlist['sim'] = cosine_similarity(nonplaylist_df.drop(['track_id'], axis = 1).values, playlist_vector.drop(labels = 'track_id').values.reshape(1, -1))[:,0]
    non_playlist_top15 = non_playlist.sort_values('sim',ascending = False).head(15)
    non_playlist_top15['url'] = non_playlist_top15['track_id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return  non_playlist_top15

In [ ]:
top15 = generate_recommendation(spotify_data, playlist_vector, nonplaylist_df)  

C:\Users\ADMIN\AppData\Local\Temp/ipykernel_18736/43927750.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_playlist['sim'] = cosine_similarity(nonplaylist_df.drop(['track_id'], axis = 1).values, playlist_vector.drop(labels = 'track_id').values.reshape(1, -1))[:,0]


In [ ]:
top15.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,sim,url
113526,Pop,Bon Iver,Rosyln,4k7x3QKrc3h3U0Viqk0uop,61,0.000049,0.000046,1.0,0.000046,0.000045,A#,0.000046,0.0,Minor,0.000045,0.000597,3/4,0.000046,1.0,https://i.scdn.co/image/ab67616d00001e028cdc33...
113563,Pop,Dreezy,Ecstasy (feat. Jeremih),5o54O3MOnMvcPTmkguYkwg,65,0.000043,0.000046,1.0,0.000044,0.000042,A#,0.000042,0.0,Minor,0.000042,0.000598,4/4,0.000045,1.0,https://i.scdn.co/image/ab67616d00001e0246b4bb...
113492,Pop,DJ Khaled,For Free,4JDZl9nKIAhAhDjw753u4X,59,0.000045,0.000050,1.0,0.000048,0.000045,A#,0.000046,0.0,Minor,0.000046,0.000608,4/4,0.000048,1.0,https://i.scdn.co/image/ab67616d00001e02782336...
108239,Pop,gianni & kyle,do u even miss me at all?,6pZrMNlxjDl216OcdBGpfJ,76,0.000042,0.000045,1.0,0.000043,0.000040,A#,0.000041,0.0,Minor,0.000041,0.000604,4/4,0.000043,1.0,https://i.scdn.co/image/ab67616d00001e023855f0...
149395,Pop,Rixton,Hotel Ceiling,2s5B2qJSK8kbFyFRiRq06b,62,0.000041,0.000044,1.0,0.000044,0.000040,A#,0.000041,0.0,Major,0.000040,0.000591,4/4,0.000043,1.0,https://i.scdn.co/image/ab67616d00001e02647377...


In [ ]:
#visualize_cover_art(top15)
top15

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,sim,url
113526,Pop,Bon Iver,Rosyln,4k7x3QKrc3h3U0Viqk0uop,61,0.000049,0.000046,1.0,0.000046,0.000045,A#,0.000046,0.0,Minor,0.000045,0.000597,3/4,0.000046,1.0,https://i.scdn.co/image/ab67616d00001e028cdc33...
113563,Pop,Dreezy,Ecstasy (feat. Jeremih),5o54O3MOnMvcPTmkguYkwg,65,0.000043,0.000046,1.0,0.000044,0.000042,A#,0.000042,0.0,Minor,0.000042,0.000598,4/4,0.000045,1.0,https://i.scdn.co/image/ab67616d00001e0246b4bb...
113492,Pop,DJ Khaled,For Free,4JDZl9nKIAhAhDjw753u4X,59,0.000045,0.000050,1.0,0.000048,0.000045,A#,0.000046,0.0,Minor,0.000046,0.000608,4/4,0.000048,1.0,https://i.scdn.co/image/ab67616d00001e02782336...
108239,Pop,gianni & kyle,do u even miss me at all?,6pZrMNlxjDl216OcdBGpfJ,76,0.000042,0.000045,1.0,0.000043,0.000040,A#,0.000041,0.0,Minor,0.000041,0.000604,4/4,0.000043,1.0,https://i.scdn.co/image/ab67616d00001e023855f0...
149395,Pop,Rixton,Hotel Ceiling,2s5B2qJSK8kbFyFRiRq06b,62,0.000041,0.000044,1.0,0.000044,0.000040,A#,0.000041,0.0,Major,0.000040,0.000591,4/4,0.000043,1.0,https://i.scdn.co/image/ab67616d00001e02647377...
150037,Pop,Travis Scott,Never Catch Me,3jg8bevUzKYONDLBBQquif,58,0.000047,0.000050,1.0,0.000050,0.000047,A#,0.000048,0.0,Minor,0.000048,0.000612,4/4,0.000048,1.0,https://i.scdn.co/image/ab67616d00001e02743317...
112593,Pop,H.E.R.,Lights On,01ZqsXtHhRtO2Z2JrvkG1w,64,0.000046,0.000045,1.0,0.000045,0.000043,A#,0.000044,0.0,Minor,0.000044,0.000580,4/4,0.000045,1.0,https://i.scdn.co/image/ab67616d00001e02b5d9c8...
112293,Pop,Prince Fox,Space (feat. Quinn XCII),1tyyQRDz6YWi3nNBP7ILfI,62,0.000040,0.000042,1.0,0.000042,0.000039,A#,0.000039,0.0,Major,0.000039,0.000592,4/4,0.000040,1.0,https://i.scdn.co/image/ab67616d00001e02a30d15...
111397,Pop,Sam Smith,Nothing Left For You,4S2WtfT03YDIlXEwSX4Tjd,61,0.000041,0.000040,1.0,0.000039,0.000038,A#,0.000039,0.0,Major,0.000038,0.000598,3/4,0.000039,1.0,https://i.scdn.co/image/ab67616d00001e02f3ea71...
149876,Pop,The Piano Guys,A Thousand Years,0E0xI4AgMNU7UkwBt3FCVN,64,0.000052,0.000050,1.0,0.000050,0.000052,A#,0.000049,0.0,Major,0.000049,0.000618,3/4,0.000049,1.0,https://i.scdn.co/image/ab67616d00001e02fce4ef...
